In [1]:
# Initial imports
import pandas as pd
from pathlib import Path
import numpy as np


# Loading Bridge Dataset

In [2]:
# Reading the csv file and create Bridge dataframe
bridge_df = pd.read_csv("Resources/NTAD_National_Bridge_Inventory_-6282134062105639862.csv", low_memory=False)
bridge_df.head()

,OBJECTID,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,...,SUBMITTED_BY,BRIDGE_CONDITION,LOWEST_RATING,DECK_AREA,STATUS,DATE,LATDD,LONGDD,x,y
0,1,1,00000000000S702,1,6,1,04007,0,09,53,...,62,F,5,145.50,ST,723,31.103039,-87.570411,-87.570411,31.103039
1,2,1,00000000000S703,1,6,1,04003,0,09,53,...,62,F,6,133.28,ST,723,31.105611,-87.569139,-87.569139,31.105611
2,3,1,0000000000M0022,1,8,1,00000,0,00,113,...,74,F,5,387.00,ST,723,32.295361,-84.977219,-84.977219,32.295361
3,4,1,000000883039900,1,4,1,00088,0,02,59,...,67,G,7,687.75,AM,1202,34.454228,-87.975517,-87.975517,34.454228
4,5,1,000001014002450,1,3,1,00101,0,02,79,...,67,F,5,15453.36,MM,403,34.813317,-87.382372,-87.382372,34.813317


In [3]:
# # Selecting bridges in NY and CA states
# bridge_NYCA_df = bridge_df.loc[(bridge_df['STATE_CODE_001'] == 36) | (bridge_df['STATE_CODE_001'] == 6),:]
# bridge_NYCA_df.head()

In [4]:
# Selecting features 
list_columns = [1,
1,
0,
0,
1,
1,
0,
0,
0,
0,
0,
1,
0,
0,
0,
1,
0,
1,
0,
0,
0,
0,
0,
1,
0,
0,
1,
1,
1,
1,
1,
1,
1,
1,
0,
1,
1,
1,
1,
0,
0,
0,
1,
1,
1,
0,
0,
0,
0,
1,
0,
0,
1,
1,
1,
1,
1,
0,
0,
1,
1,
1,
1,
1,
0,
0,
0,
1,
1,
1,
1,
1,
1,
1,
0,
0,
1,
1,
1,
0,
1,
1,
0,
0,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
0,
0,
0,
1,
1,
0,
0,
1,
0,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
0,
1,
1,
1,
0,
1,
0,
0,
0,
0,
]

In [5]:
# converitng to a boolean list
bool_sequence = [bool(value) for value in list_columns]
bool_sequence[:5]

#  Selecting informative columns
bridge_reduced_df = bridge_df.loc[:, bool_sequence]
bridge_reduced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621581 entries, 0 to 621580
Data columns (total 80 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   OBJECTID                 621581 non-null  int64  
 1   STATE_CODE_001           621581 non-null  int64  
 2   ROUTE_PREFIX_005B        621581 non-null  int64  
 3   SERVICE_LEVEL_005C       621581 non-null  int64  
 4   FEATURES_DESC_006A       621565 non-null  object 
 5   MIN_VERT_CLR_010         621581 non-null  float64
 6   BASE_HWY_NETWORK_012     602800 non-null  float64
 7   TOLL_020                 621581 non-null  int64  
 8   FUNCTIONAL_CLASS_026     621581 non-null  int64  
 9   YEAR_BUILT_027           621581 non-null  int64  
 10  TRAFFIC_LANES_ON_028A    621581 non-null  int64  
 11  TRAFFIC_LANES_UND_028B   621581 non-null  int64  
 12  ADT_029                  621581 non-null  int64  
 13  YEAR_ADT_030             621580 non-null  float64
 14  DESI

 # Cleaning data

In [6]:
bridge_cleaned_df2 = bridge_reduced_df.copy()

In [7]:
# Uniforming data 
bridge_cleaned_df2['FEATURES_DESC_006A'] = bridge_cleaned_df2['FEATURES_DESC_006A'].str.lower()

# Selecting features near the bridge
river_rows = bridge_cleaned_df2[bridge_cleaned_df2['FEATURES_DESC_006A'].str.contains('river', na=False)]
branch_rows = bridge_cleaned_df2[bridge_cleaned_df2['FEATURES_DESC_006A'].str.contains('branch', na=False)]
stream_rows = bridge_cleaned_df2[bridge_cleaned_df2['FEATURES_DESC_006A'].str.contains('stream', na=False)]
bayou_rows = bridge_cleaned_df2[bridge_cleaned_df2['FEATURES_DESC_006A'].str.contains('bayou', na=False)]
wash_rows = bridge_cleaned_df2[bridge_cleaned_df2['FEATURES_DESC_006A'].str.contains('wash', na=False)]
waterway_rows = bridge_cleaned_df2[bridge_cleaned_df2['FEATURES_DESC_006A'].str.contains('waterway', na=False)]
draw_rows = bridge_cleaned_df2[bridge_cleaned_df2['FEATURES_DESC_006A'].str.contains('draw', na=False)]

pattern_canal = '|'.join(['drainage','drain','ditch','canal'])
canal_rows = bridge_cleaned_df2[bridge_cleaned_df2['FEATURES_DESC_006A'].str.contains(pattern_canal, na=False)]
pattern_creek = '|'.join(['creek','cr','ck'])
creek_rows = bridge_cleaned_df2[bridge_cleaned_df2['FEATURES_DESC_006A'].str.contains(pattern_creek, na=False)]

# Reducing the number of Variables

bridge_cleaned_df2.loc[river_rows.index, 'FEATURES_DESC_006A'] = 'river'
bridge_cleaned_df2.loc[creek_rows.index, 'FEATURES_DESC_006A'] = 'creek'
bridge_cleaned_df2.loc[canal_rows.index, 'FEATURES_DESC_006A'] = 'canal'
bridge_cleaned_df2.loc[bayou_rows.index, 'FEATURES_DESC_006A'] = 'bayou'
bridge_cleaned_df2.loc[branch_rows.index, 'FEATURES_DESC_006A'] = 'branch'
bridge_cleaned_df2.loc[stream_rows.index, 'FEATURES_DESC_006A'] = 'stream'
bridge_cleaned_df2.loc[wash_rows.index, 'FEATURES_DESC_006A'] = 'wash'
bridge_cleaned_df2.loc[waterway_rows.index, 'FEATURES_DESC_006A'] = 'waterway'
bridge_cleaned_df2.loc[draw_rows.index, 'FEATURES_DESC_006A'] = 'draw'

pattern_road = '|'.join(['creek','river', 'canal','bayou','branch','stream','wash','waterway','draw'])
road_rows = bridge_cleaned_df2[~bridge_cleaned_df2['FEATURES_DESC_006A'].str.contains(pattern_road, na=False)]
bridge_cleaned_df2.loc[road_rows.index, 'FEATURES_DESC_006A'] = 'road'

bridge_cleaned_df2['FEATURES_DESC_006A'].value_counts()

FEATURES_DESC_006A
creek       289616
road        165617
river        53849
canal        38884
branch       32271
stream       16821
draw          8386
wash          7446
bayou         6879
waterway      1812
Name: count, dtype: int64

In [8]:
# converting YEAR_BUILT_027 column to Bridge age column
year_built = bridge_cleaned_df2['YEAR_BUILT_027']
bridge_age = [2024-year for year in year_built]
bridge_cleaned_df2['bridge_age'] = bridge_age
del bridge_cleaned_df2['YEAR_BUILT_027']

# converting YEAR_OF_IMP_097 column to Bridge age column
year_reconstructed = bridge_cleaned_df2['YEAR_RECONSTRUCTED_106']
new_age = []

# Loop through index-value pairs of the Series
for idx, year_r in year_reconstructed.items():
    if not pd.isna(year_r) and year_r > 1:
        age_r = 2024 - year_r
        index_r = idx
        if age_r > 0 :
            bridge_reconstructed = {'index': index_r, 'bridge_age': age_r}
            new_age.append(bridge_reconstructed)
# Bridge Age replacement
for replacement_data in new_age:
    bridge_cleaned_df2.loc[replacement_data['index'], 'bridge_age'] = replacement_data['bridge_age']

del bridge_cleaned_df2['YEAR_RECONSTRUCTED_106']
bridge_cleaned_df2['bridge_age'].head(10)

0     25
1     22
2     82
3     50
4     62
5     66
6    126
7    124
8     85
9    124
Name: bridge_age, dtype: int64

In [9]:
# transforming future ADT data
adt_year =  bridge_cleaned_df2['YEAR_OF_FUTURE_ADT_115'] -bridge_cleaned_df2['YEAR_ADT_030']
adt_year[adt_year<0] = 0                        
future_adt_value =  bridge_cleaned_df2['FUTURE_ADT_114'] - bridge_cleaned_df2['ADT_029']
# perc_adt = round((future_adt_value / bridge_cleaned_df2['ADT_029'])*100,2)

# creating new variabes
# bridge_cleaned_df2['change_perc_adt'] = perc_adt
bridge_cleaned_df2['future_adt_estimation'] = adt_year

# deleting redundant variables
del bridge_cleaned_df2['YEAR_OF_FUTURE_ADT_115']
del bridge_cleaned_df2['YEAR_ADT_030']

In [10]:
#checking Nan Valus for each columns
nan_counts = bridge_cleaned_df2.isna().sum()
print(nan_counts[nan_counts>=1])

BASE_HWY_NETWORK_012        18781
TRANSITIONS_036B                1
OPR_RATING_METH_063            25
OPERATING_RATING_064         1014
APPR_ROAD_EVAL_072              1
IMP_LEN_MT_076             169588
FRACTURE_LAST_DATE_093A    604730
UNDWATER_LAST_DATE_093B    590251
SPEC_LAST_DATE_093C        603240
BRIDGE_IMP_COST_094        209105
ROADWAY_IMP_COST_095       218380
TOTAL_IMP_COST_096         223271
YEAR_OF_IMP_097            481985
PERCENT_ADT_TRUCK_109       28603
PIER_PROTECTION_111        524960
MIN_NAV_CLR_MT_116         351863
future_adt_estimation           1
dtype: int64


In [11]:
# Converting to 0 nan values
bridge_cleaned_df2['BASE_HWY_NETWORK_012'].fillna(0, inplace=True)
bridge_cleaned_df2['OPERATING_RATING_064'].fillna(0, inplace=True)
bridge_cleaned_df2['IMP_LEN_MT_076'].fillna(0, inplace=True)
bridge_cleaned_df2['BRIDGE_IMP_COST_094'].fillna(0, inplace=True)
bridge_cleaned_df2['ROADWAY_IMP_COST_095'].fillna(0, inplace=True)
bridge_cleaned_df2['TOTAL_IMP_COST_096'].fillna(0, inplace=True)
bridge_cleaned_df2['PERCENT_ADT_TRUCK_109'].fillna(0, inplace=True)
bridge_cleaned_df2['PIER_PROTECTION_111'].fillna(0, inplace=True)
bridge_cleaned_df2['MIN_NAV_CLR_MT_116'].fillna(0, inplace=True)
# bridge_cleaned_df2['change_perc_adt'].fillna(0, inplace=True)

# removing row for nan values
bridge_cleaned_df2.dropna(subset=['TRANSITIONS_036B'], inplace=True)
bridge_cleaned_df2.dropna(subset=['APPR_ROAD_EVAL_072'], inplace=True)
bridge_cleaned_df2.dropna(subset=['future_adt_estimation'], inplace=True)

In [12]:
# Deleting incomplete columns
del bridge_cleaned_df2['FRACTURE_LAST_DATE_093A']
del bridge_cleaned_df2['UNDWATER_LAST_DATE_093B']
del bridge_cleaned_df2['SPEC_LAST_DATE_093C']
del bridge_cleaned_df2['YEAR_OF_IMP_097']
del bridge_cleaned_df2['OPR_RATING_METH_063']

In [13]:
#checking Nan Valus for each columns
nan_counts = bridge_cleaned_df2.isna().sum()
print(nan_counts[nan_counts>=1])

Series([], dtype: int64)


In [14]:
# Look at UNDWATER_LOOK_SEE_092B value counts for binning
underwater_count = bridge_cleaned_df2['UNDWATER_LOOK_SEE_092B'].value_counts()
# You may find it helpful to look at CLASSIFICATION value counts >1
underwater_count_h1 =underwater_count[underwater_count>0]

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace2 = list(underwater_count[underwater_count<100].index)


# Replace in dataframe
for cls in classifications_to_replace2:
    bridge_cleaned_df2['UNDWATER_LOOK_SEE_092B'] = bridge_cleaned_df2['UNDWATER_LOOK_SEE_092B'].replace(cls,"Other")
    
# Check to make sure binning was successful
bridge_cleaned_df2['UNDWATER_LOOK_SEE_092B'].value_counts()

UNDWATER_LOOK_SEE_092B
N        523083
N00       66922
Y60       20756
Y24        6511
Y48        2400
Y36         863
Y72         374
Y12         272
Y23         213
Other       185
Name: count, dtype: int64

In [15]:
# Look at SPEC_INSPECT_092C value counts for binning
spec_count = bridge_cleaned_df2['SPEC_INSPECT_092C'].value_counts()
# You may find it helpful to look at CLASSIFICATION value counts >1
spec_count_h1 =spec_count[spec_count>0]

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(spec_count[spec_count<100].index)

# Replace in dataframe
for cls in classifications_to_replace:
    bridge_cleaned_df2['SPEC_INSPECT_092C'] = bridge_cleaned_df2['SPEC_INSPECT_092C'].replace(cls,"Other")
    
# Check to make sure binning was successful
bridge_cleaned_df2['SPEC_INSPECT_092C'].value_counts()

SPEC_INSPECT_092C
N        536699
N00       66365
Y12        9203
Y24        3218
Y00        2614
Y06        1587
Y48         934
Y03         306
Y60         181
Y96         169
Other       157
Y72         146
Name: count, dtype: int64

In [16]:
# Look at FRACTURE_092A value counts for binning
fracture_count = bridge_cleaned_df2['FRACTURE_092A'].value_counts()
# You may find it helpful to look at CLASSIFICATION value counts >1
fracture_count_h1 =fracture_count[fracture_count>0]

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(fracture_count[fracture_count<100].index)

# Replace in dataframe
for cls in classifications_to_replace:
    bridge_cleaned_df2['FRACTURE_092A'] = bridge_cleaned_df2['FRACTURE_092A'].replace(cls,"Other")
    
# Check to make sure binning was successful
bridge_cleaned_df2['FRACTURE_092A'].value_counts()

FRACTURE_092A
N        536197
N00       68508
Y24       13985
Y12        2774
Other       115
Name: count, dtype: int64

# Exporting Bridge Dataset

In [19]:
bridge_cleaned_df2.to_csv('Resources/bridge_testing_dataset.csv', index=False)